# SPE DSEATS AFRICA - DATATHON 2025
**Overview**

The Society of Petroleum Engineer (SPE) Data Science and Engineering Analytics Technical Section (DSEATS) Africa Datathon Challenge presents an exciting opportunity for participants to harness the power of machine learning to address real-world challenges in the oil and gas industry. Organized by SPE DSEATS Africa Region, this challenge aims to leverage historical production data to accurately classify a group of 20 wells based on their observed performance trends.

**Packages Required**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier

# LOAD THE DATASET

In [4]:
data_df_well = pd.read_csv("data/spe_africa_dseats_datathon_2025_wells_dataset.csv")
data_df_well.head()

,PROD_DATE,WELL_NAME,ON_STREAM_HRS,BOTTOMHOLE_FLOWING_PRESSURE (PSI),DOWNHOLE_TEMPERATURE (deg F),ANNULUS_PRESS (PSI),CHOKE_SIZE (%),WELL_HEAD_PRESSURE (PSI),WELL_HEAD_TEMPERATURE (deg F),CUMULATIVE_OIL_PROD (STB),CUMULATIVE_FORMATION_GAS_PROD (MSCF),CUMULATIVE_TOTAL_GAS_PROD (MSCF),CUMULATIVE_WATER_PROD (BBL)
0,15-Feb-14,Well_#1,0.0,"4,050",189.866,0,1.17951,482.46,50.864,0,0,0,0
1,16-Feb-14,Well_#1,0.0,"3,961",189.945,0,2.99440,328.601,47.668,0,0,0,0
2,17-Feb-14,Well_#1,0.0,"3,961",190.004,0,1.90349,387.218,48.962,0,0,0,0
3,18-Feb-14,Well_#1,0.0,"3,964",190.020,0,0.00000,308.98,46.636,0,0,0,0
4,19-Feb-14,Well_#1,0.0,"3,965",190.107,0,30.20760,196.057,47.297,0,0,0,0


<a href="https://colab.research.google.com/github/nkosi-da-p/SPE_DATATHON/blob/main/DataPhandas_DSEATS_Africa_2025_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>